In [1]:
import numpy as np 
import pandas as pd 
import gc

pd.options.mode.chained_assignment = None  # default='warn'
#import matplotlib.pyplot as plt
#import seaborn as sns
df_cal = pd.read_csv('calendar.csv')
df_eval = pd.read_csv('sales_train_evaluation.csv')
df_price = pd.read_csv('sell_prices.csv')
#df_sample_output = pd.read_csv('sample_submission.csv')

In [2]:
df_cal

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [3]:
hol_list = df_cal['event_name_1'].unique()[1:]
wkd_list = ['Saturday','Sunday']
df_cal['is_holiday_1'] = df_cal['event_name_1'].apply(lambda x : 1 if x in hol_list else 0 )
df_cal['is_holiday_2'] = df_cal['event_name_1'].apply(lambda x : 1 if x in hol_list else 0 )
df_cal['is_holiday'] = df_cal[['is_holiday_1','is_holiday_2']].max(axis=1)
df_cal['is_weekend'] = df_cal['weekday'].apply(lambda x : 1 if x in wkd_list else 0 )
df_cal = df_cal.drop(['weekday', 'wday', 'month', 'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], axis='columns')

In [4]:
df_cal

,date,wm_yr_wk,d,snap_CA,snap_TX,snap_WI,is_holiday_1,is_holiday_2,is_holiday,is_weekend
0,2011-01-29,11101,d_1,0,0,0,0,0,0,1
1,2011-01-30,11101,d_2,0,0,0,0,0,0,1
2,2011-01-31,11101,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,d_4,1,1,0,0,0,0,0
4,2011-02-02,11101,d_5,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,d_1965,0,1,1,0,0,0,0
1965,2016-06-16,11620,d_1966,0,0,0,0,0,0,0
1966,2016-06-17,11620,d_1967,0,0,0,0,0,0,0
1967,2016-06-18,11621,d_1968,0,0,0,0,0,0,1


In [5]:
df_eval

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [6]:
# for all FOODS_1
eva_all = df_eval[df_eval['dept_id'] == 'FOODS_1']
eva_all = eva_all.melt(['id','item_id','dept_id','cat_id','store_id','state_id'],var_name = 'd', value_name = 'sales')
eva_all = pd.merge(eva_all, df_cal, how = 'left', on = 'd')
eva_all = pd.merge(eva_all, df_price, how = 'left', on = ['item_id', 'wm_yr_wk', 'store_id'])
eva_all = eva_all.drop(['is_holiday_1','is_holiday_2'], axis =1)
eva_all
############################################################ Times cost: 107 seconds

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,0,0,0,0,1,2.00
1,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,0,0,0,0,1,7.88
2,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,0,0,0,0,1,2.88
3,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,0,0,0,0,1,NaN
4,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,0,0,0,0,1,2.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192555,FOODS_1_215_WI_3_evaluation,FOODS_1_215,FOODS_1,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,0,0,0,0,1,1.84
4192556,FOODS_1_216_WI_3_evaluation,FOODS_1_216,FOODS_1,FOODS,WI_3,WI,d_1941,2,2016-05-22,11617,0,0,0,0,1,5.28
4192557,FOODS_1_217_WI_3_evaluation,FOODS_1_217,FOODS_1,FOODS,WI_3,WI,d_1941,1,2016-05-22,11617,0,0,0,0,1,3.54
4192558,FOODS_1_218_WI_3_evaluation,FOODS_1_218,FOODS_1,FOODS,WI_3,WI,d_1941,20,2016-05-22,11617,0,0,0,0,1,0.98


In [7]:
len(eva_all['item_id'].unique())

216

In [8]:
cat_encode = {"HOBBIES":1,"HOUSEHOLD":2,"FOODS":3}
state_encode = {"CA":1,"TX":2,"WI":3}
dept_encode = {'HOBBIES_1':11, 'HOBBIES_2':12, 
               'HOUSEHOLD_1':21, 'HOUSEHOLD_2':22, 
               'FOODS_1':31,'FOODS_2':32, 'FOODS_3':33}
store_encode = {'CA_1':11, 'CA_2':12, 'CA_3':13, 'CA_4':14, 
                'TX_1':21, 'TX_2':22, 'TX_3':23, 
                'WI_1':31, 'WI_2':32, 'WI_3':33}
def data_encode(df):
    #other_columns = ['cat_id','state_id']
    # use only numerical columns
    df['state_id'] = df['state_id'].replace(state_encode)
    df['cat_id'] = df["cat_id"].replace(cat_encode)
    df['dept_id'] = df["dept_id"].replace(dept_encode)
    df['store_id'] = df["store_id"].replace(store_encode)
    import re
    df['d'] = df['d'].apply(lambda x : int(re.search(r'd_(\d+)', x).group(1)))
    # calendar [["wm_yr_wk", "wday", "month", "year",'d','snap_CA','snap_TX',"snap_WI"]]
    return df

def trans_str(ori_str):
    trans_str = ori_str.replace('_', '')
    trans_str = trans_str.replace('HOBBIES', '1')
    trans_str = trans_str.replace('HOUSEHOLD', '2')
    trans_str = trans_str.replace('FOODS', '3')
    trans_str = trans_str.replace('CA', '1')
    trans_str = trans_str.replace('TX', '2')
    trans_str = trans_str.replace('WI', '3')
    trans_str = trans_str.replace('evaluation', '')
    return trans_str
eva_all_encode = data_encode(eva_all)
eva_all_encode['item_id'] = eva_all_encode['item_id'].apply(trans_str)
eva_all_encode['item_id'] = eva_all_encode['item_id'].astype(int)
eva_all_encode['id'] = eva_all_encode['id'].apply(trans_str)
eva_all_encode['id'] = eva_all_encode['id'].astype(int)
uni_val = eva_all_encode['id'].unique()
mapping = {val: idx for idx, val in enumerate(uni_val)}
eva_all_encode['id'] = np.vectorize(mapping.get)(eva_all_encode['id'])
eva_all_encode = eva_all_encode.drop(['date','wm_yr_wk'],axis=1)#,'id'

eva_all_encode
############################################## Time cost : 121 seconds

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price
0,0,31001,31,3,11,1,1,3,0,0,0,0,1,2.00
1,1,31002,31,3,11,1,1,0,0,0,0,0,1,7.88
2,2,31003,31,3,11,1,1,0,0,0,0,0,1,2.88
3,3,31004,31,3,11,1,1,0,0,0,0,0,1,NaN
4,4,31005,31,3,11,1,1,3,0,0,0,0,1,2.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192555,2155,31215,31,3,33,3,1941,0,0,0,0,0,1,1.84
4192556,2156,31216,31,3,33,3,1941,2,0,0,0,0,1,5.28
4192557,2157,31217,31,3,33,3,1941,1,0,0,0,0,1,3.54
4192558,2158,31218,31,3,33,3,1941,20,0,0,0,0,1,0.98


In [9]:
len(eva_all['id'].unique())

2160

In [10]:
eva_all_encode.isna().sum()

id                 0
item_id            0
dept_id            0
cat_id             0
store_id           0
state_id           0
d                  0
sales              0
snap_CA            0
snap_TX            0
snap_WI            0
is_holiday         0
is_weekend         0
sell_price    601342
dtype: int64

In [11]:
eva_f1 = eva_all_encode#.drop('id',axis=1)
eva_f1

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price
0,0,31001,31,3,11,1,1,3,0,0,0,0,1,2.00
1,1,31002,31,3,11,1,1,0,0,0,0,0,1,7.88
2,2,31003,31,3,11,1,1,0,0,0,0,0,1,2.88
3,3,31004,31,3,11,1,1,0,0,0,0,0,1,NaN
4,4,31005,31,3,11,1,1,3,0,0,0,0,1,2.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192555,2155,31215,31,3,33,3,1941,0,0,0,0,0,1,1.84
4192556,2156,31216,31,3,33,3,1941,2,0,0,0,0,1,5.28
4192557,2157,31217,31,3,33,3,1941,1,0,0,0,0,1,3.54
4192558,2158,31218,31,3,33,3,1941,20,0,0,0,0,1,0.98


In [14]:
del eva_all_encode
gc.collect()

0

In [15]:
# Level data generation:
col_1_L1 = ['sell_price','sales', 'item_id','store_id', 'state_id'] #,'dept_id', 'cat_id' 'id',
col_1_L2 = ['sell_price','sales', 'item_id','store_id'] #,'dept_id', 'cat_id', 'state_id' 'id',
col_1_L3 = ['sell_price','sales', 'item_id'] #,'dept_id', 'cat_id', 'store_id'            'id', 
col_1_L4 = ['sell_price','sales', ] #'id',

col_2_L1 = ['is_weekend','is_holiday','snap_CA','snap_TX','snap_WI']
col_2_L2 = ['is_weekend','is_holiday','snap_CA','snap_TX','snap_WI']
col_2_L3 = ['is_weekend','is_holiday','snap_CA','snap_TX','snap_WI']
col_2_L4 = ['is_weekend','is_holiday','snap_CA','snap_TX','snap_WI']

col_1_L = [col_1_L1, 
           col_1_L2,
           col_1_L3,
           col_1_L4]
col_2_L = [col_2_L1, 
           col_2_L2,
           col_2_L3,
           col_2_L4]
columns_name = ['d','item_id', 'store_id', 'state_id', 'snap_CA', 'snap_TX', 'snap_WI', 'is_holiday','is_weekend', 'sell_price','sales']#'dept_id', 'cat_id','id',

In [16]:
# generrate the hierarchical time series based on the 4 levels
def levelG(df,col_1, col_2, level_n, columns):
    df_list = []
    col = []
    for i in [0,1,2,3]:
        col = list(set(col_1[0]) - set(col_1[i]))
        col.append('d')
        sum_L = df.groupby(col)[col_1[i]].sum().reset_index()
        mea_L = df.groupby(col)[col_2[i]].mean().reset_index().iloc[:,i+1:]
        level = pd.concat([sum_L,mea_L],axis=1)[columns]
        df_list.append(level)
    return df_list

In [17]:
eva_list = levelG(eva_f1, col_1_L, col_2_L, 4, columns_name)

In [18]:
eva_list[0]

,d,item_id,store_id,state_id,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price,sales
0,1,67198180,45792,4104,0.0,0.0,0.0,0.0,1.0,2988.53,2343
1,2,67198180,45792,4104,0.0,0.0,0.0,0.0,1.0,2988.53,2216
2,3,67198180,45792,4104,0.0,0.0,0.0,0.0,0.0,2988.53,1657
3,4,67198180,45792,4104,1.0,1.0,0.0,0.0,0.0,2988.53,1508
4,5,67198180,45792,4104,1.0,0.0,1.0,0.0,0.0,2988.53,1209
...,...,...,...,...,...,...,...,...,...,...,...
1936,1937,67198180,45792,4104,0.0,0.0,0.0,0.0,0.0,7367.44,3073
1937,1938,67198180,45792,4104,0.0,0.0,0.0,0.0,0.0,7367.44,3251
1938,1939,67198180,45792,4104,0.0,0.0,0.0,0.0,0.0,7367.44,3735
1939,1940,67198180,45792,4104,0.0,0.0,0.0,0.0,1.0,7368.77,4195


In [19]:
eva_list[1]

,d,item_id,store_id,state_id,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price,sales
0,1,26879272,10800,1,0.0,0.0,0.0,0.0,1.0,1198.90,1157
1,2,26879272,10800,1,0.0,0.0,0.0,0.0,1.0,1198.90,1142
2,3,26879272,10800,1,0.0,0.0,0.0,0.0,0.0,1198.90,787
3,4,26879272,10800,1,1.0,1.0,0.0,0.0,0.0,1198.90,743
4,5,26879272,10800,1,1.0,0.0,1.0,0.0,0.0,1198.90,729
...,...,...,...,...,...,...,...,...,...,...,...
5818,1937,20159454,20736,3,0.0,0.0,0.0,0.0,0.0,2280.30,950
5819,1938,20159454,20736,3,0.0,0.0,0.0,0.0,0.0,2280.30,1048
5820,1939,20159454,20736,3,0.0,0.0,0.0,0.0,0.0,2280.30,1106
5821,1940,20159454,20736,3,0.0,0.0,0.0,0.0,1.0,2280.68,1120


In [20]:
eva_list[3]########  FOODS_1_219_WI_3 : 3_1_219_3_3

,d,item_id,store_id,state_id,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price,sales
0,1,31001,11,1,0.0,0.0,0.0,0.0,1.0,2.00,3
1,2,31001,11,1,0.0,0.0,0.0,0.0,1.0,2.00,0
2,3,31001,11,1,0.0,0.0,0.0,0.0,0.0,2.00,0
3,4,31001,11,1,1.0,1.0,0.0,0.0,0.0,2.00,1
4,5,31001,11,1,1.0,0.0,1.0,0.0,0.0,2.00,4
...,...,...,...,...,...,...,...,...,...,...,...
4192555,1937,31219,33,3,0.0,0.0,0.0,0.0,0.0,2.24,0
4192556,1938,31219,33,3,0.0,0.0,0.0,0.0,0.0,2.24,0
4192557,1939,31219,33,3,0.0,0.0,0.0,0.0,0.0,2.24,0
4192558,1940,31219,33,3,0.0,0.0,0.0,0.0,1.0,2.24,0


In [22]:
eva_list[3]['id'].unique()#total 216, absent : 31007, 31100, and  

array([   0,    1,    2, ..., 2157, 2158, 2159])

In [27]:
#### collect_xy for TOP LEVEL (L1)
def collect_xy(df,colname_1,colname_2,x_window:int = 60, y_window: int = 28):
    # for generating x
    df_ = df.copy()
    for i in range(x_window):
        df_[f'x_{i+1}DaysAgo'] = df_[colname_1].shift(i+1)
        
    # for generating y
    for i in range(y_window):
        df_[f"y_{i+1}DaysF"] = df_[colname_2].shift(-i-1)
    df_ = df_.iloc[x_window : -y_window,:]
    return df_

### cominbe_df_xy for other levels
'''
def combine_df_xy(df,colname_3, col_s):
    from tqdm import tqdm
    uni_id = df[colname_3].unique()
    df_c = pd.DataFrame()
    for i in tqdm(range(len(uni_id))):
        df_e = df[df[colname_3]==uni_id[i]]
        df_d = collect_xy(df_e ,col_s ,col_s)
        df_c = pd.concat([df_c,df_d])
    return df_c'''

'\ndef combine_df_xy(df,colname_3, col_s):\n    from tqdm import tqdm\n    uni_id = df[colname_3].unique()\n    df_c = pd.DataFrame()\n    for i in tqdm(range(len(uni_id))):\n        df_e = df[df[colname_3]==uni_id[i]]\n        df_d = collect_xy(df_e ,col_s ,col_s)\n        df_c = pd.concat([df_c,df_d])\n    return df_c'

In [42]:
### cominbe_df_xy for other levels
def combine_df_xy(df,group_n, col_list):
    from tqdm import tqdm
    uni_id = df[col_list[0]].unique()
    df_c = pd.DataFrame()
    if group_n == 1:
        for i in tqdm(range(len(uni_id))):
            df_e = df[df[col_list[0]]==uni_id[i]]
            df_e = collect_xy(df_e ,col_list[-1] ,col_list[-1])
            df_c = pd.concat([df_c,df_e])
    elif group_n == 2 : # This number is the amount of level names required for us to disaggregate the hierarchical structure
        for i in tqdm(range(len(uni_id))):
            uni_id_2 = df[col_list[1]].unique()
            for x in range(len(uni_id_2)):
                      df_e = df[(df[col_list[0]]==uni_id[i]) & (df[col_list[1]]==uni_id_2[x])]
                      df_e = collect_xy(df_e ,col_list[-1] ,col_list[-1])
                      df_c = pd.concat([df_c,df_e])
    return df_c

In [43]:
l1 = collect_xy(eva_list[0],'sales','sales')
l2 = combine_df_xy(eva_list[1],1,['state_id', 'sales'])
l3 = combine_df_xy(eva_list[2],1,['store_id', 'sales'])
l4 = combine_df_xy(eva_list[3],2,['store_id','item_id','sales'])
#827

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [13:47<00:00, 82.76s/it]


In [44]:
l4

,d,item_id,store_id,state_id,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price,...,y_19DaysF,y_20DaysF,y_21DaysF,y_22DaysF,y_23DaysF,y_24DaysF,y_25DaysF,y_26DaysF,y_27DaysF,y_28DaysF
60,61,31001,11,1,0.0,0.0,0.0,0.0,0.0,2.00,...,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
61,62,31001,11,1,0.0,0.0,0.0,0.0,0.0,2.00,...,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
62,63,31001,11,1,1.0,1.0,0.0,0.0,0.0,2.00,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
63,64,31001,11,1,1.0,0.0,1.0,0.0,1.0,2.00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
64,65,31001,11,1,1.0,1.0,1.0,0.0,1.0,2.00,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192527,1909,31219,33,3,0.0,0.0,0.0,0.0,0.0,2.24,...,1.0,2.0,5.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0
4192528,1910,31219,33,3,0.0,0.0,0.0,0.0,0.0,2.24,...,2.0,5.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0
4192529,1911,31219,33,3,0.0,0.0,0.0,0.0,0.0,2.24,...,5.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4192530,1912,31219,33,3,0.0,0.0,0.0,0.0,1.0,2.24,...,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
l3

,d,item_id,store_id,state_id,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price,...,y_19DaysF,y_20DaysF,y_21DaysF,y_22DaysF,y_23DaysF,y_24DaysF,y_25DaysF,y_26DaysF,y_27DaysF,y_28DaysF
60,61,6719818,11,1,0.0,0.0,0.0,0.0,0.0,339.62,...,141.0,169.0,157.0,183.0,358.0,452.0,186.0,121.0,130.0,114.0
61,62,6719818,11,1,0.0,0.0,0.0,0.0,0.0,339.62,...,169.0,157.0,183.0,358.0,452.0,186.0,121.0,130.0,114.0,125.0
62,63,6719818,11,1,1.0,1.0,0.0,0.0,0.0,339.62,...,157.0,183.0,358.0,452.0,186.0,121.0,130.0,114.0,125.0,188.0
63,64,6719818,11,1,1.0,0.0,1.0,0.0,1.0,349.41,...,183.0,358.0,452.0,186.0,121.0,130.0,114.0,125.0,188.0,205.0
64,65,6719818,11,1,1.0,1.0,1.0,0.0,1.0,349.41,...,358.0,452.0,186.0,121.0,130.0,114.0,125.0,188.0,205.0,209.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19377,1909,6719818,33,3,0.0,0.0,0.0,0.0,0.0,760.06,...,325.0,302.0,308.0,350.0,336.0,423.0,386.0,271.0,279.0,264.0
19378,1910,6719818,33,3,0.0,0.0,0.0,0.0,0.0,760.06,...,302.0,308.0,350.0,336.0,423.0,386.0,271.0,279.0,264.0,338.0
19379,1911,6719818,33,3,0.0,0.0,0.0,0.0,0.0,760.06,...,308.0,350.0,336.0,423.0,386.0,271.0,279.0,264.0,338.0,285.0
19380,1912,6719818,33,3,0.0,0.0,0.0,0.0,1.0,760.06,...,350.0,336.0,423.0,386.0,271.0,279.0,264.0,338.0,285.0,315.0


In [48]:
def dataSplit(df):
    x_train, y_train = df[ df['d']<1913 ].iloc[:,:-28], df[df['d']<1913].iloc[:,-28:]
    x_test, y_test   = df[df['d'] >= 1913].iloc[:,:-28], df[df['d'] >= 1913].iloc[:,-28:]
    return x_train,x_test,y_train,y_test

def Concat(df1,df2):
    df = pd.concat([df1,df2], axis=0)
    return df

In [64]:
hts = [l1, l2, l3, l4]
a1,b1,c1,d1 = pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()#x_train, x_test, y_train, y_test
for i in range(len(hts)) :
    a, b, c, d = dataSplit(hts[i])
    a1, b1, c1, d1 = Concat(a1, a), Concat(b1, b), Concat(c1, c), Concat(d1, d)
del a,b,c,d
gc.collect()

0

In [162]:
a1

,d,item_id,store_id,state_id,snap_CA,snap_TX,snap_WI,is_holiday,is_weekend,sell_price,...,x_51DaysAgo,x_52DaysAgo,x_53DaysAgo,x_54DaysAgo,x_55DaysAgo,x_56DaysAgo,x_57DaysAgo,x_58DaysAgo,x_59DaysAgo,x_60DaysAgo
60,61,67198180,45792,4104,0.0,0.0,0.0,0.0,0.0,3405.80,...,1586.0,1925.0,2235.0,1903.0,1897.0,1209.0,1508.0,1657.0,2216.0,2343.0
61,62,67198180,45792,4104,0.0,0.0,0.0,0.0,0.0,3405.80,...,1785.0,1586.0,1925.0,2235.0,1903.0,1897.0,1209.0,1508.0,1657.0,2216.0
62,63,67198180,45792,4104,1.0,1.0,0.0,0.0,0.0,3405.80,...,1737.0,1785.0,1586.0,1925.0,2235.0,1903.0,1897.0,1209.0,1508.0,1657.0
63,64,67198180,45792,4104,1.0,0.0,1.0,0.0,1.0,3486.51,...,2048.0,1737.0,1785.0,1586.0,1925.0,2235.0,1903.0,1897.0,1209.0,1508.0
64,65,67198180,45792,4104,1.0,1.0,1.0,0.0,1.0,3486.51,...,2349.0,2048.0,1737.0,1785.0,1586.0,1925.0,2235.0,1903.0,1897.0,1209.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192526,1908,31219,33,3,0.0,0.0,0.0,0.0,0.0,2.24,...,4.0,5.0,2.0,0.0,3.0,0.0,3.0,6.0,4.0,3.0
4192527,1909,31219,33,3,0.0,0.0,0.0,0.0,0.0,2.24,...,3.0,4.0,5.0,2.0,0.0,3.0,0.0,3.0,6.0,4.0
4192528,1910,31219,33,3,0.0,0.0,0.0,0.0,0.0,2.24,...,1.0,3.0,4.0,5.0,2.0,0.0,3.0,0.0,3.0,6.0
4192529,1911,31219,33,3,0.0,0.0,0.0,0.0,0.0,2.24,...,2.0,1.0,3.0,4.0,5.0,2.0,0.0,3.0,0.0,3.0


In [163]:
x_train, x_test, y_train, y_test= a1.iloc[:,4:],b1.iloc[:,4:],c1,d1

In [67]:
y_test

,y_1DaysF,y_2DaysF,y_3DaysF,y_4DaysF,y_5DaysF,y_6DaysF,y_7DaysF,y_8DaysF,y_9DaysF,y_10DaysF,...,y_19DaysF,y_20DaysF,y_21DaysF,y_22DaysF,y_23DaysF,y_24DaysF,y_25DaysF,y_26DaysF,y_27DaysF,y_28DaysF
1912,2841.0,2860.0,2799.0,3184.0,3800.0,4415.0,4285.0,3506.0,4020.0,3602.0,...,4130.0,4257.0,4225.0,3264.0,3119.0,3073.0,3251.0,3735.0,4195.0,4031.0
1912,1417.0,1275.0,1352.0,1440.0,1506.0,1960.0,1914.0,1535.0,1614.0,1598.0,...,1931.0,1854.0,1820.0,1408.0,1316.0,1374.0,1364.0,1631.0,2095.0,2041.0
3853,653.0,683.0,703.0,752.0,964.0,946.0,964.0,849.0,1046.0,773.0,...,1022.0,924.0,1025.0,824.0,797.0,749.0,839.0,998.0,980.0,962.0
5794,771.0,902.0,744.0,992.0,1330.0,1509.0,1407.0,1122.0,1360.0,1231.0,...,1177.0,1479.0,1380.0,1032.0,1006.0,950.0,1048.0,1106.0,1120.0,1028.0
1912,294.0,259.0,268.0,328.0,387.0,455.0,403.0,262.0,332.0,272.0,...,412.0,435.0,415.0,277.0,302.0,397.0,330.0,310.0,427.0,334.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4184767,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,...,3.0,2.0,0.0,0.0,4.0,0.0,3.0,1.0,1.0,0.0
4186708,2.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,...,3.0,1.0,2.0,4.0,0.0,0.0,1.0,1.0,0.0,2.0
4188649,1.0,0.0,3.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,...,2.0,4.0,1.0,5.0,2.0,2.0,1.0,0.0,1.0,1.0
4190590,14.0,13.0,16.0,9.0,12.0,15.0,20.0,11.0,20.0,38.0,...,17.0,21.0,20.0,20.0,2.0,6.0,25.0,8.0,14.0,20.0


In [165]:
from sklearn.multioutput import RegressorChain
from sklearn.metrics import mean_squared_error as mse
def get_lgb():
    from lightgbm import LGBMRegressor as lgbr
    return lgbr(
        n_jobs = -1,
        boosting_type= 'gbdt',
        objective = "regression",
        metric = "rmse",
        num_leaves =64,
        learning_rate = 0.1,
        # 0.05 - 74~75
        #bagging_fraction = 0.7,
        #feature_fraction = 0.5,
        #bagging_frequency = 5,
        #bagging_seed = 42,
        verbosity = 1,
        #seed= 42,
        force_col_wise=True)
#for model_name, get_model in [("LightGB",get_lgb)]:
    #print(f"Model: {model_name}")
l1 = RegressorChain(get_lgb())
l1.fit(x_train, y_train)

[LightGBM] [Info] Total Bins 15820
[LightGBM] [Info] Number of data points in the train set: 4026248, number of used features: 67
[LightGBM] [Info] Start training from score 4.943179
[LightGBM] [Info] Total Bins 16075
[LightGBM] [Info] Number of data points in the train set: 4026248, number of used features: 68
[LightGBM] [Info] Start training from score 4.944538
[LightGBM] [Info] Total Bins 16330
[LightGBM] [Info] Number of data points in the train set: 4026248, number of used features: 69
[LightGBM] [Info] Start training from score 4.945578
[LightGBM] [Info] Total Bins 16585
[LightGBM] [Info] Number of data points in the train set: 4026248, number of used features: 70
[LightGBM] [Info] Start training from score 4.946270
[LightGBM] [Info] Total Bins 16840
[LightGBM] [Info] Number of data points in the train set: 4026248, number of used features: 71
[LightGBM] [Info] Start training from score 4.947360
[LightGBM] [Info] Total Bins 17095
[LightGBM] [Info] Number of data points in the tra

RegressorChain(base_estimator=LGBMRegressor(force_col_wise=True, metric='rmse',
                                            n_jobs=-1, num_leaves=64,
                                            objective='regression',
                                            verbosity=1))

In [166]:
from sklearn.metrics import mean_squared_error as mse
train_pred = l1.predict(x_train)
test_pred = l1.predict(x_test)
print(f"Train score: {mse(train_pred, y_train, squared=True)}")
print(f"Test score: {mse(test_pred, y_test, squared=True)}")

Train score: 55.05877765904048
Test score: 249.5901185608129


In [72]:
len(test_pred)

2174

In [ ]:
# this is for h = 1 
'''
def forecast_mase(forecast, truth):
    mase = []
    from sklearn.metrics import mean_absolute_error as mae
    for i in range(int(len(forecast)/28)):
        mase.append(mae(forecast[i*28:(i+1)*28], truth[i*28:(i+1)*28])/(truth[0:28].diff().abs()[1:].mean()).iloc[0])
    return mase'''

In [167]:
def forecast_mase(forecast_1, truth_1):
    mase = []
    forecast = np.array(forecast_1)
    truth = np.array(truth_1)
    from sklearn.metrics import mean_absolute_error as mae
    for i in range(len(forecast)):
        truth_diff = np.abs(truth[i, 1:] - truth[i, :-1])
        mean_diff = truth_diff.mean()
        if mean_diff == 0:
            mase.append(1)
        else:
            mase.append(mae(forecast[i, :], truth[i, :]) / mean_diff)
    return mase


In [170]:
mase_base = forecast_mase(test_pred, y_test)
mase_base

[1.2296099245265815,
 1.0824106178083166,
 1.6667640102286319,
 1.3989111357732507,
 0.6079710288912129,
 0.8285792150345686,
 1.023827386749365,
 1.040473730156454,
 1.2065613516276872,
 1.06548905019961,
 1.2079466621838313,
 0.8516372646035565,
 1.3467533509139422,
 1.4617148246005718,
 0.8255452075482843,
 0.7672622543290101,
 1.123801738893809,
 1.1624545775789543,
 0.7648782285047796,
 0.7692599994666217,
 1.9228958413204706,
 0.8574162552438805,
 1.7864074167605106,
 1.0769740734715305,
 0.8243372904115093,
 2.4172975277897266,
 0.6692142703011122,
 0.5856053898059548,
 0.9134441938659721,
 0.6840982917711644,
 0.8107407560398966,
 0.8382926504119251,
 0.7485688465480677,
 3.4066910653738027,
 3.41538540320846,
 0.7662492279805848,
 0.6959611545639063,
 1.0855707863621145,
 0.682025110362676,
 0.6878552449559423,
 1.0512999398986074,
 1.2803113479115344,
 0.6368686600506064,
 0.8747324244096372,
 0.748434995512353,
 6.140084325575685,
 1.489326727810538,
 0.9819526972320853,
 0.

In [171]:
#### mase for the hierarchical time series
np.mean(mase_base)

1.3158538702379967

In [172]:
################################# SUMMING MATRIX FOR BU, TD, AND MINT
def SMatrix(level_number, bottom_level_number, eva_list_ts):
    l = []
    i = 0
    while i <= level_number:
        if i == 0:  # （1，2160）
            # for top level
            a = np.ones(bottom_level_number).reshape(1,2160)
            l.append(a)
            
        elif i == 1:  #（3，2160）
            # for l2
            l.append([])
            o = [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                 [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
                 [0, 0, 0, 0, 0, 0, 0, 1, 1, 1]]
            for x1 in range(len(eva_list_ts[i]['state_id'].unique())):
                r = (o[x1] * (bottom_level_number // len(o) + 1))[:bottom_level_number]
                l[i].append(r)
        elif i == 2:  #（10，2160）
            # for l3 store
            l.append([])
            for x2 in range(len(eva_list_ts[i]['store_id'].unique())):
                o = np.zeros(bottom_level_number)
                o[x2::9] = 1
                l[i].append(o)
        elif i == 3:  # （2160，2160）
            # for l4
            l.append(np.eye(bottom_level_number))
        else:
            break
        i += 1
    S = np.concatenate((l[0], l[1], l[2], l[3]), axis=0)
    return S

In [173]:
S_Matrix = SMatrix(4,2160, eva_list)

In [174]:
np.shape(S_Matrix)

(2174, 2160)

In [175]:
######## G matrix for BU
g1 = np.zeros((2160,14))#（ bottom_level , except_bottom level）
g2 = np.eye(2160)# bottom level
G_bu = np.concatenate((g1,g2), axis=1)
G_bu

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [176]:
np.shape(G_bu)

(2160, 2174)

In [177]:
SG_bu = np.dot(S_Matrix, G_bu)

In [178]:
############# for all 3 reconciliation methods
y_base_frec = test_pred

In [179]:
y_coh_frec_bu = np.dot(SG_bu, y_base_frec)
y_coh_frec_bu

array([[2.87651594e+03, 2.88886796e+03, 3.05448467e+03, ...,
        3.23890896e+03, 3.68239120e+03, 3.61020697e+03],
       [1.15285566e+03, 1.16127878e+03, 1.22888075e+03, ...,
        1.27958351e+03, 1.45188589e+03, 1.41643607e+03],
       [8.76852706e+02, 8.73162061e+02, 9.31618346e+02, ...,
        1.00057453e+03, 1.13372807e+03, 1.11445414e+03],
       ...,
       [1.44018815e+00, 1.61382193e+00, 1.61714190e+00, ...,
        9.89314339e-01, 1.43821784e+00, 1.65357173e+00],
       [8.06389118e+00, 8.17645894e+00, 7.46221391e+00, ...,
        9.64958238e+00, 1.02114286e+01, 1.01243739e+01],
       [2.46507755e+00, 1.72148187e+00, 1.86736819e+00, ...,
        1.31485159e+00, 1.68587989e+00, 2.49754941e+00]])

In [117]:
# BU's MASE
re_bu = forecast_mase(y_coh_frec_bu, y_test)
np.mean(re_bu)

1.3027955646304088

In [180]:
# BU's MASE
re_bu = forecast_mase(y_coh_frec_bu, y_test)
np.mean(re_bu)

1.3184602381260797

In [181]:
###########################TD
# G_Matrix_TD
def G_Matrix_TD(bottom_level, top_level, ts_length, col_bo, col_to, ts_amount):
    from typing import Optional
    from tqdm import tqdm
    prop_mean = []
    n = int(len(bottom_level[col_bo])/ts_length)
    for i in tqdm(range(n)):
        prop = []
        #prop = np.divide(np.array(bottom_level[col_bo][i*ts_length : (i+1)*ts_length]), np.array(top_level[col_to]))
        #, out=np.zeros_like(vector1), where=(vector2 != 0)
        with np.errstate(divide='ignore', invalid='ignore'):
            prop =  (bottom_level[col_bo].iloc[i*ts_length : (i+1)*ts_length].values/top_level[col_to].values)
        prop_mean.append(prop)
    prop_mean = np.nan_to_num(prop_mean)
    prop_mean = np.mean(prop_mean,axis = 1).reshape(-1,1)
    g2 = np.zeros((n, ts_amount-1))
    G = np.concatenate((prop_mean, g2), axis=1)
    return G

In [182]:
G_td = G_Matrix_TD(eva_list[3], eva_list[0], 1941, 'sales', 'sales', 2174)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2160/2160 [00:00<00:00, 14975.03it/s]


In [183]:
SG_td = np.dot(S_Matrix, G_td)
SG_td

array([[9.98969603e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.01042555e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.06295927e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [7.13887221e-04, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.36664829e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.24399064e-04, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [184]:
y_coh_frec_td = np.dot(SG_td, y_base_frec)
y_coh_frec_td

array([[2.51515594e+03, 2.56858750e+03, 2.69101942e+03, ...,
        3.67579707e+03, 4.38172442e+03, 3.67600742e+03],
       [1.00972498e+03, 1.03117542e+03, 1.08032647e+03, ...,
        1.47567158e+03, 1.75907050e+03, 1.47575602e+03],
       [7.71176637e+02, 7.87559388e+02, 8.25098465e+02, ...,
        1.12704297e+03, 1.34348867e+03, 1.12710747e+03],
       ...,
       [1.79738971e+00, 1.83557316e+00, 1.92306590e+00, ...,
        2.62681121e+00, 3.13128354e+00, 2.62696154e+00],
       [1.09941297e+01, 1.12276872e+01, 1.17628557e+01, ...,
        1.60674689e+01, 1.91531848e+01, 1.60683883e+01],
       [2.07563093e+00, 2.11972529e+00, 2.22076216e+00, ...,
        3.03344932e+00, 3.61601545e+00, 3.03362292e+00]])

In [185]:
re_td = forecast_mase(y_coh_frec_td, y_test)
print(np.mean(re_td))

1.0592137853357302


In [186]:
np.shape(S_Matrix)

(2174, 2160)

In [187]:
########  MinT‘s G matrix ############
### estimate Wh matrix at first # size (2174*2174)
def W_matrix_G(forecast_1, truth_1):
    forecast = np.array(forecast_1)
    truth = np.array(truth_1)
    residuals_ = []
    for i in range(int(len(forecast))):
        res = np.mean(np.dot(truth[i,:]-forecast[i,:],truth[i,:]-forecast[i,:]))
        residuals_.append(res)
    W = np.diag(residuals_)
    return W, residuals_

In [188]:
W , residuals =W_matrix_G(test_pred, y_test.values)

In [189]:
residuals
np.shape(residuals)
np.shape(W)

(2174, 2174)

In [190]:
np.shape(S_Matrix.T)

(2160, 2174)

In [191]:
np.linalg.inv(W)

array([[1.17439225e-07, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 6.38327475e-07, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 9.17165423e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.83932421e-02, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 4.03873215e-04, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 9.41812207e-03]])

In [192]:
G_mint = np.linalg.inv(S_Matrix.T@np.linalg.inv(W)@S_Matrix)@S_Matrix.T@np.linalg.inv(W)

In [193]:
G_mint

array([[ 4.46206321e-06,  2.83893164e-05, -2.48999557e-06, ...,
        -8.78527949e-07,  2.89935016e-06, -2.76267946e-07],
       [ 1.74389013e-06,  9.87771264e-06, -3.74041947e-07, ...,
        -1.11034720e-06,  3.21630626e-07, -8.65426983e-07],
       [ 2.64526690e-06,  1.59748697e-05, -1.07736543e-06, ...,
        -1.07427297e-06,  1.13348745e-06, -7.10522733e-07],
       ...,
       [ 5.29694380e-06, -1.26190785e-06, -1.21405601e-06, ...,
         9.99746076e-01, -3.05697562e-05, -3.23430929e-05],
       [ 1.80077189e-04, -2.07165606e-04, -1.94421930e-04, ...,
        -1.39221148e-03,  9.84540244e-01, -1.35706585e-03],
       [ 9.70524363e-06, -3.40033631e-06, -3.79441027e-06, ...,
        -6.31648574e-05, -5.81944621e-05,  9.99710985e-01]])

In [157]:
np.shape(G_mint)

(2160, 2174)

In [194]:
SG_mint = S_Matrix@ G_mint
SG_mint

array([[ 3.31971919e-02,  7.79990718e-02,  6.45417006e-02, ...,
         8.29603304e-01,  6.19285022e-01,  7.78318904e-01],
       [ 1.43502370e-02,  9.55192312e-02, -1.04863057e-02, ...,
        -3.63615501e-02, -1.31074789e-01, -5.01698324e-02],
       [ 8.26429686e-03, -7.29824403e-03,  8.25103101e-02, ...,
        -2.43472176e-02, -8.56135740e-02, -3.89637662e-02],
       ...,
       [ 5.29694380e-06, -1.26190785e-06, -1.21405601e-06, ...,
         9.99746076e-01, -3.05697562e-05, -3.23430929e-05],
       [ 1.80077189e-04, -2.07165606e-04, -1.94421930e-04, ...,
        -1.39221148e-03,  9.84540244e-01, -1.35706585e-03],
       [ 9.70524363e-06, -3.40033631e-06, -3.79441027e-06, ...,
        -6.31648574e-05, -5.81944621e-05,  9.99710985e-01]])

In [195]:
y_coh_frec_mint = SG_mint@y_base_frec
y_coh_frec_mint

array([[2.69276541e+03, 2.68093585e+03, 2.79490060e+03, ...,
        3.13759788e+03, 3.56975039e+03, 3.41219453e+03],
       [1.07530361e+03, 1.07133179e+03, 1.11242042e+03, ...,
        1.23981638e+03, 1.41028825e+03, 1.33025508e+03],
       [8.17504374e+02, 8.10721283e+02, 8.57410244e+02, ...,
        9.64562791e+02, 1.08801555e+03, 1.04830136e+03],
       ...,
       [1.42666959e+00, 1.60099444e+00, 1.60488897e+00, ...,
        9.91631520e-01, 1.44034349e+00, 1.64699341e+00],
       [4.55409224e+00, 4.33685603e+00, 3.42489063e+00, ...,
        6.50418903e+00, 7.17281762e+00, 6.31911409e+00],
       [2.45602399e+00, 1.71387258e+00, 1.85391804e+00, ...,
        1.32110017e+00, 1.70186291e+00, 2.50238806e+00]])

In [196]:
re_mint = forecast_mase(y_coh_frec_mint, y_test)
print(np.mean(re_mint))

1.3183330221882803


In [161]:
######### feature involves d, item_id, store_id, state_id
print("lgbm's forecasts:", np.mean(mase_base))
print("lgbm with BU's forecasts:", np.mean(re_bu))
print("lgbm with TD's forecasts:", np.mean(re_td))
print("lgbm with MinT's forecasts:", np.mean(re_mint))

lgbm's forecasts: 1.2984727648944563
lgbm with BU's forecasts: 1.3027955646304088
lgbm with TD's forecasts: 1.0716987729093137
lgbm with MinT's forecasts: 1.3051139364946365


In [197]:
######### feature without d, item_id, store_id, state_id
print("lgbm's forecasts:", np.mean(mase_base))
print("lgbm with BU's forecasts:", np.mean(re_bu))
print("lgbm with TD's forecasts:", np.mean(re_td))
print("lgbm with MinT's forecasts:", np.mean(re_mint))

lgbm's forecasts: 1.3158538702379967
lgbm with BU's forecasts: 1.3184602381260797
lgbm with TD's forecasts: 1.0592137853357302
lgbm with MinT's forecasts: 1.3183330221882803
